In [1]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import openai
import threading

# Directly set the API key
openai.api_key = 'sk-proj-IuNe5dmEfg3JjN0vUa7YT3BlbkFJ9li48prvLGsegUVTRyba'

# Define a dictionary with weather data (temperature for each location)
weather_data_dict = {
    'new york': 25,
    'mumbai': 34,
    'jaipur': 21,
    'manali': 5
}

def get_weather(location):
    # Look up the weather data based on the location (case-neutral)
    location = location.lower()
    temperature = weather_data_dict.get(location)
    if temperature is None:
        return json.dumps({
            "location": location,
            "temperature": "N/A",
            "description": "Data not available"
        }, indent=4)
    
    # Provide a default description
    description = "clear sky"  # You can adjust this as needed
    weather_data = {
        "location": location,
        "temperature": f"{temperature}°C",
        "description": description
    }
    return json.dumps(weather_data, indent=4)

def get_assistant_response(messages):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=150
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {str(e)}"

def handle_user_request(user_input):
    if "weather" in user_input.lower():
        location = extract_location(user_input)
        weather_json = get_weather(location)
        weather_data = json.loads(weather_json)
        return (f"The current temperature in {weather_data['location']} is {weather_data['temperature']} "
                f"with {weather_data['description']}.")
    else:
        # Create a conversation history for the assistant
        messages = [
            {"role": "system", "content": "You are a knowledgeable assistant capable of answering a wide range of questions."},
            {"role": "user", "content": user_input}
        ]
        return get_assistant_response(messages)

def extract_location(user_input):
    user_input = user_input.lower()
    location_start = user_input.find("in") + 3
    if location_start == -1:
        return "London"  # Default location if no city is mentioned
    return user_input[location_start:].strip()

def handle_input():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        
        response = handle_user_request(user_input)
        print("Assistant:", response)

def threaded_input():
    input_thread = threading.Thread(target=handle_input)
    input_thread.start()
    input_thread.join()  # Wait for the thread to finish

# Start the threaded input
threaded_input()


You: hi
Assistant: Hello! How can I assist you today?
You: weather in new york
Assistant: The current temperature in new york is 25°C with clear sky.
You: hi , how are you ? when did india gain its independence and from whome ?
Assistant: Hello! I'm here and ready to help you. India gained its independence from British colonial rule on August 15, 1947. This day is celebrated annually as Independence Day in India.
